# Segment and quantify puncta in individual cells

Detect and segment puncta and analyze their size and intensity. Calculate puncta statistics per cell nucleus.

## Requirements
- A folder with images that should be analyzed.  All z-layers for a specific sample must be combined into a single file. To combine z-layers and channels, run [run_images_to_stack.ipynb](run_images_to_stack.ipynb). 

- To compute puncta statistics per cell, cell segmentation should be provided as an additional channel. To segment cells/nuclei, run [run_cell_segmentation.ipynb](run_cell_segmentation.ipynb).

<hr style="height:2px;">

## Config

<hr style="height:2px;">

### The following code imports and declares functions used for the processing:

In [1]:
#################################
#  Don't modify the code below  #
#################################

import json
import os
from punctatools.lib.segment import segment_puncta_batch
from punctatools.lib.quantify import quantify_batch
from punctatools.lib.utils import load_parameters, save_parameters

2022-01-24 11:39:17,059 [INFO] WRITING LOG OUTPUT TO /home/amedyukh/.cellpose/run.log


### Specify the parameters for segmentation

<hr style="height:0.5px;">

#### Option 1 (preferred)

Specify the parameter file to read the parameters (`parameter_file`)

To set up the parameters and generate the parameter file, run the notebook [setup_puncta_analysis.ipynb](setup_puncta_analysis.ipynb)

<hr style="height:0.5px;">

#### Option 2

Specify the parameter values:

`input_dir`: folder with images to be analyzed

`output_dir`: folder to save the result of the puncta analysis

`puncta_segm_dir`: subfolder for puncta segmentation

`puncta_stat_dir`: subfolder for individual puncta measurements

`cell_stat_dir`: subfolder for individual cell measurements

`puncta_channels`: list of channel indices, starting form 0, where puncta should be detected.

`cell_segmentation`: True of False. If True, use the last channel of the input image as cell/nuclei mask.

`minsize_um`: minimal sigma for the Laplacian of Gaussian detection (microns); default is 0.2

`maxsize_um`: maximal sigma for the Laplacian of Gaussian detection (microns); default is 2

`num_sigma`: number of sigma values for the Laplacian of Gaussian detection; default is 5

`overlap`: a value between 0 and 1; if two blobs overlaps by a fraction greater than this value, the smaller blob is eliminated; default is 1 (blobs are removed only if overlapping completely)

`threshold_detection`: threshold for detecting LoG blobs. The absolute lower bound for scale space maxima. Local maxima smaller than thresh are ignored. Reduce this to detect blobs with less intensities. Should be close to 0 and can be both positive and negative.

`threshold_background`: threshold used to post-filter puncta in cells with diffuse signal. This threshold is provided relative to the median GFP intensity inside cells (e.g, `threshold_background` = 2 will result in all puncta with intensity lower than two median GPF (background) intensities being removed). Set to 0 to keep all puncta.

`global_background`: If True, the background value is calculated globally as the `global_background_percentile` of all cells.(Default is True)

`global_background_percentile`: Percentile (between 0 and 100) of cell background values to calculate the global background value. (Default is 95).

`background_percentile`: Percentile (between 0 and 100) of image intensity inside cell to calculate the background value. (Default is 50 (median)).

`threshold_segmentation`: Threshold for puncta segmentation. The way the threshold is applied is determined by `segmentation_mode`. For mode 0, choose values in the order of 0.001; for mode 1, choose values in the order of 50; for mode 2, choose values in the order of 3. Reduce to detect more/larger puncta, increase to detect fewer/smaller puncta

`segmentation_mode`: 0, 1, or 2. Determines the mode how `threshold_segmentation` is applied; 0: apply absolute threshold in LoG space; 1: apply threshold relative to background in LoG space; 2: apply threshold relative to the background in image intensity space.

`remove_out_of_cell`: If True, remove all puncta (parts) that are not inside cells/nuclei.

`maxrad_um`: If not None, remove puncta with a radius larger than this value. Set to None to keep all detected puncta.


`n_jobs`: number of processes to run in parallel. Set according to your workstation resources. Decrease if it runs out of memory. (Default is 8)

`channel_names`: list of names for the channels in the original image

In [2]:
parameter_file = 'parameters.json'

# input_dir = "../test_output/cells"
# output_dir = "../test_output/puncta_analysis"

# puncta_segm_dir = 'puncta'
# puncta_stat_dir = 'puncta_stats'
# cell_stat_dir = 'cell_stats'

# puncta_channels = [1, 2]

# cell_segmentation = True

# minsize_um = 0.2 
# maxsize_um = 2  
# num_sigma = 5  
# overlap = 1  

# threshold_detection = 0.001 

# threshold_background = 3
# global_background = True
# global_background_percentile = 95
# background_percentile = 50

# threshold_segmentation = 50  
# segmentation_mode = 1
# remove_out_of_cell = True
# maxrad_um = None

# n_jobs=8

# channel_names = ['ch0', 'ch1', 'ch2']

<hr style="height:2px;">

## Processing

<hr style="height:2px;">


### The following code loads the parameters 

In [3]:
#################################
#  Don't modify the code below  #
#################################

param_keys = [
    'puncta_segm_dir',
    'puncta_quant_dir',
    'roi_quant_dir',
    'puncta_channels',
    'cell_segmentation',
    'minsize_um',
    'maxsize_um',
    'num_sigma',
    'overlap', 
    'threshold_detection', 
    'threshold_background', 
    'global_background', 
    'global_background_percentile', 
    'background_percentile', 
    'threshold_segmentation',
    'segmentation_mode',
    'remove_out_of_cell', 
    'maxrad_um', 
    'n_jobs',
    'channel_names' 
]
param_matches = dict(output_dir='puncta_analysis_dir')
kwargs = load_parameters(vars(), param_keys, param_matches)
if kwargs['cell_segmentation']:
    param_matches['input_dir'] = 'cell_segmentation_dir'
else:
    param_matches['input_dir'] = 'converted_data_dir'
kwargs = load_parameters(vars(), param_keys, param_matches)
os.makedirs(kwargs['output_dir'], exist_ok=True)
params = save_parameters(kwargs, 
                         os.path.join(kwargs['output_dir'], parameter_file))

kwargs

{'puncta_segm_dir': 'puncta_segm',
 'puncta_quant_dir': 'puncta_quants',
 'roi_quant_dir': 'cell_quants',
 'puncta_channels': [1, 2],
 'cell_segmentation': False,
 'minsize_um': 0.2,
 'maxsize_um': 2,
 'num_sigma': 5,
 'overlap': 1,
 'threshold_detection': [0.001, 0.0001],
 'threshold_background': [3, 1.5],
 'global_background': False,
 'global_background_percentile': 95,
 'background_percentile': 50,
 'threshold_segmentation': [0.001, 0.0005],
 'segmentation_mode': 0,
 'remove_out_of_cell': False,
 'maxrad_um': None,
 'n_jobs': 8,
 'channel_names': ['ch0', 'ch1', 'ch3'],
 'output_dir': '/tmp/pycharm_project_989/test_output/puncta_analysis',
 'input_dir': '/tmp/pycharm_project_989/test_output/cells'}

### The following code segments puncta in all image in the input folder

In [4]:
#################################
#  Don't modify the code below  #
#################################

channel_names = kwargs.pop('channel_names')
puncta_segm_dir = kwargs.pop('puncta_segm_dir')
roi_quant_dir = kwargs.pop('roi_quant_dir')
puncta_quant_dir = kwargs.pop('puncta_quant_dir')

segm_kwargs = kwargs.copy()
segm_kwargs['output_dir'] = os.path.join(segm_kwargs['output_dir'], puncta_segm_dir)
segment_puncta_batch(parallel=True, process_name='Segment puncta', **segm_kwargs)

Run Segment puncta


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 129.33it/s]


Segment puncta done


### The following code quantifies all images

In [5]:
#################################
#  Don't modify the code below  #
#################################

quantify_batch(input_dir=segm_kwargs['output_dir'],
               output_dir_puncta=os.path.join(kwargs['output_dir'], puncta_quant_dir),
               output_dir_cells=os.path.join(kwargs['output_dir'], roi_quant_dir),
               parallel=True, n_jobs=kwargs['n_jobs'],
               channel_names=channel_names,
               puncta_channels=kwargs['puncta_channels'],
               process_name='Quantify puncta')

Run Quantify puncta


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 122.45it/s]
/research/sharedresources/cbi/public/conda_envs/punctatools/lib/python3.9/site-packages/punctatools/lib/quantify.py:124: RuntimeWarning: invalid value encountered in long_scalars
  coloc = np.sum((overlap[cur_cell_pix] > 0)*1) / np.sum(union[cur_cell_pix])
/research/sharedresources/cbi/public/conda_envs/punctatools/lib/python3.9/site-packages/punctatools/lib/quantify.py:124: RuntimeWarning: invalid value encountered in long_scalars
  coloc = np.sum((overlap[cur_cell_pix] > 0)*1) / np.sum(union[cur_cell_pix])
/research/sharedresources/cbi/public/conda_envs/punctatools/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/research/sharedre

Quantify puncta done


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 373.46it/s]
